In [1]:
import pandas as pd
import numpy as np
import math
import plotly.express as px
import seaborn as sns
from plotly.subplots import make_subplots

### To show all columns

In [2]:
pd.set_option('display.max_columns', None)

### Read data

In [3]:
data = pd.read_csv('RiyadhVillasAqar.csv',low_memory=False)

FileNotFoundError: [Errno 2] No such file or directory: 'RiyadhVillasAqar.csv'

In [ ]:
df=data.copy()

In [ ]:
df

### drop uneeded columns

In [ ]:
df.columns

In [ ]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [ ]:
df.columns

In [ ]:
df.info()

### data describe

In [ ]:
df.describe().T

### show unique value for every column

In [ ]:
for i in df.columns:
    print(i," : " ,df[i].unique())

# Handel duplicated unique values 

In [ ]:
def convert(x):
    try:
        new=float(x)
        new=int(new)
        new=str(new)
        return new
    except:
        new=x
        return new
df['apartments']=df['apartments'].apply(convert)
df['lounges']=df['lounges'].apply(convert)

In [ ]:
df['apartments'].unique()

In [ ]:
df['lounges'].unique()

# Handel missing value

In [ ]:
df.isna().sum()[df.isna().sum()>0]

### Street width column

In [ ]:
nul=df[df['streetWidth'].isna()]
nul

In [ ]:
nul['front'].value_counts()

In [ ]:
px.histogram(df,x='streetWidth')

In [ ]:
def null_streatWidth(row):
    mean=df[(df['neighbourhood'] == row['neighbourhood']) & ((df['square price'] <= row['square price']+300) & ((df['square price'] >= row['square price']-300))) & (df['streetWidth'].isna()==False) & ((df['front'] == row['front']))].mean()
    return mean
df.loc[nul.index,'streetWidth'] = df.loc[nul.index,:].apply(null_streatWidth,axis=1)

In [ ]:
px.histogram(df,x='streetWidth')

### square price  and price columns

check if square price calculated correctlly

In [ ]:
def trueSquer(row):
    squer=row['price']/row['space']
    return squer
df['true squre']=df.apply(trueSquer,axis=1)

In [ ]:
df[round(df['square price']) != round(df['true squre'])]

In [ ]:
nul=df[df['square price'].isna()]
nul

In [ ]:
px.histogram(df,x='square price')

In [ ]:
px.histogram(df,x='price')

In [ ]:
def null_squer_price(row):
    mean=df[(df['neighbourhood'] == row['neighbourhood']) & (df['space'] == row['space']) & (df['elevator'] == row['elevator'])]['square price'].mean()
    return mean
df.loc[nul.index,'square price'] = df.loc[nul.index,:].apply(null_squer_price,axis=1)
df.loc[nul.index,'price'] = df.loc[nul.index,'square price']*df.loc[nul.index,'space']

### lounges column

In [ ]:
df['lounges'].unique()

In [ ]:
px.histogram(df,x='lounges')

In [ ]:
df['lounges'].fillna(0,inplace=True)

In [ ]:
px.histogram(df,x='lounges')

# handel outliers

In [ ]:
df.info()

In [ ]:
px.box(df[['rooms','propertyAge']])

In [ ]:
px.box(df['price'])

In [ ]:
px.box(df['square price'])

In [ ]:
px.box(df['space'])

### check duplicated

In [ ]:
df.duplicated().sum()

### data shape

In [ ]:
df.shape

### correlation matrix

In [ ]:
df.corr()

# enter your code

In [ ]:
df1 = pd.DataFrame(columns=['location','propertyAge','avg price'])
location= df['location'].unique()
property_age=df['propertyAge'].unique()
for i in location:
    for j in property_age: 
        m1 =(df['propertyAge']==j) & (df['location']==i)
        list1=[i,j,df.loc[m1,'price'].mean()]
        df1.loc[len(df1)] =list1

In [ ]:
df1

In [ ]:
fig = px.bar(x=df1['propertyAge'],y=df1['avg price'],color=df1['location'])
fig.update_layout(
    xaxis_title="Property Age", yaxis_title="Average price")

fig.show()

In [ ]:
df

In [ ]:
import math
twi_dim=[[1,1,1],[1,1,0],[1,0,1],[1,0,0],[0,1,1],[0,1,0],[0,0,1],[0,0,0]]
Availability_list = ['with (basement,pool,elevator)','with (basement,pool)','with (basement,elevator)','with (basement)','with (pool,elevator)','with (pool)','with (elevator)','without (basement,pool,elevator)']
df2= pd.DataFrame(columns=['location','Availability','average price','number of property'])
list2=[]
for i in location:
    for j in range(len(twi_dim)):
        dfnew =df[(df['basement']==twi_dim[j][0]) &(df['pool']==twi_dim[j][1]) & (df['elevator']==twi_dim[j][2]) &(df['location']==i)]['price']
        mean1=dfnew.mean()
        if not math.isnan(mean1):
            list2=[i,Availability_list[j],int(mean1),len(dfnew)]
        else:
            list2=[i,Availability_list[j],mean1,len(dfnew)]
        df2.loc[len(df2)] =list2

In [ ]:
df2

In [ ]:
fig = px.sunburst(df2, path=['Availability', 'location', 'average price'], values='number of property')
fig.show()

In [ ]:
fig = px.bar(x=df2['Availability'],y=df2['average price'],color=df2['location'],barmode="group")
fig.update_layout(
    xaxis_title="Availability of pool, basement and elevator", yaxis_title="Average price")

fig.show()